In [18]:
import pandas as pd
import re
import enchant
import json
import statistics
from unidecode import unidecode
from collections import Counter, defaultdict
from itertools import chain

In [19]:
class ChineseFinder(object):
    
    def __init__(self):
        
        self.ench_dic = enchant.Dict("en_US")  # english spellcheck
        # chinese names and surnames
        self.chinese_names = set(chain.from_iterable([unidecode(part).replace("-"," ").split() 
                                    for full_name in pd.read_csv("names_cn.txt", header=None)[0].tolist() 
                                                      for part in full_name.split()]))
        self.korean_names = set(chain.from_iterable([unidecode(part).replace("-"," ").split() 
                                    for full_name in pd.read_csv("names_ko.txt", header=None)[0].tolist() 
                                                     for part in full_name.split()]))
        # vietnamese last names, cover 90% of population (wikipedia)
        self.vietnamese_names = set(pd.read_csv("names_vn.txt", header=None)[0].tolist())
        self.ticketek_customers = pd.read_csv("ticketek_customers.csv.gz", dtype=str).drop("middle_name", axis=1)
        self.ticketek_customers["full_name"] = self.ticketek_customers["name"] + " " + self.ticketek_customers["last_name"]
        # hypocorisms; we just make a set of these and don't care what names they relate to
        self.hypoc_dict = json.load(open("hypocorisms.json", "r")) 
        self.hypocs = {hyp for full_name in self.hypoc_dict
                               for hyp in self.hypoc_dict[full_name]}
        self.chinese_letter_stats = defaultdict()
        self.chinese_cust_ids = set()
        # a mask to exclude de longhi, te vroomer, etc that have parts like in chinese names
        self.non_chinese_mask = re.compile('\s+[ltodaelsn]{2,3}\s+\w{5,}(\s|$)', re.ASCII)
        # customer data frame will be processed by chunks specified below
        self.CHUNK_SIZE = 100000    # in rows
        self.FULL_CHUNKS, self.ROWS_LEFT = divmod(len(self.ticketek_customers), self.CHUNK_SIZE)
    
    def _last_n_letters(self, names, n):  
        
        last_letters = [w[-n:] for w in names if len(w[-n:]) == n]
        most_frequent = statistics.mode(last_letters)
              
        return (set(last_letters), most_frequent)
        
    def get_name_stats(self):
        
        for nletters in range(1,4):
            lst, top = self._last_n_letters(self.chinese_names, nletters)
            self.chinese_letter_stats["last" + str(nletters)] = lst
            self.chinese_letter_stats["toplast" + str(nletters)] = top
        
        self.chinese_letter_stats["max_len"] = len(max(self.chinese_names, key=len))
        self.chinese_letter_stats["avg_len"] = round(statistics.mean([len(name) for name in self.chinese_names]),0)
        
        return self
    
    def _is_hypoc(self, st):
        return 1 if (set(str(st).split()) & self.hypocs) else 0
    
    def _is_chinese(self, st):
        return 1 if (set(str(st).split()) & self.chinese_names) else 0
    
    def _is_korean(self, st):
        return 1 if (set(str(st).split()) & self.korean_names) else 0
    
    def _is_vietnamese(self, st):
        return 1 if (set(str(st).split()) & self.vietnamese_names) else 0
    
    def _is_english_word(self, st):
        return int(self.ench_dic.check(st))
    
    def is_likely_chinese(self, st):
        if (self._is_chinese(st) and 
            (not self._is_korean(st)) and 
                (not self._is_hypoc(st)) and 
                    (all([not self._is_english_word(part) for part in str(st).split()]))):
            return 1
        else:
            return 0
    
    def get_top_popular_last_names(self, n):
        
        self.most_popular_last_names = sorted([(k,v) for k, v in Counter(self.chinese_df["last_name"].tolist()).items()], 
                                              key=lambda x: x[1], reverse=True)
        return self
        
    def find_chinese_in_dataframe(self):
        
        print("searching for chinese among all Ticketek customers...")

        for i in range(self.FULL_CHUNKS + 1):
            
            LAST_ONE = i*self.CHUNK_SIZE + self.CHUNK_SIZE if i < self.FULL_CHUNKS else i*self.CHUNK_SIZE + self.ROWS_LEFT
            sub_df = self.ticketek_customers.iloc[i*self.CHUNK_SIZE:LAST_ONE,:]
            
            self.chinese_cust_ids.update(set(sub_df.loc[sub_df["full_name"].apply(self.is_likely_chinese) & 
                       sub_df["last_name"].apply(lambda _: (sum([str(_).endswith(w) for w in self.chinese_letter_stats["last2"]]) > 0)
                                                and (not self.non_chinese_mask.search(str(_))) and
                                                (not self._is_vietnamese(str(_)))),
                       "cust_id"].tolist()))

            if (i%20 == 0) or (i == self.FULL_CHUNKS-1):
                print("ids found so far: {}".format(len(self.chinese_cust_ids)))
                
        self.chinese_df = self.ticketek_customers.loc[self.ticketek_customers.cust_id.isin(self.chinese_cust_ids),:]
        
        return self
        

In [20]:
if __name__ == "__main__":
    
    cf = ChineseFinder().get_name_stats()

In [21]:
cf.find_chinese_in_dataframe()

searching for chinese among all Ticketek customers...
ids found so far: 513
ids found so far: 11686
ids found so far: 19374
ids found so far: 27016
ids found so far: 38545
ids found so far: 49869
ids found so far: 63247
ids found so far: 79877
ids found so far: 96647
ids found so far: 121552
ids found so far: 146203
ids found so far: 147323
